In [1]:
# This code is originally obtained from https://github.com/paul-rottger/hatecheck-experiments and modified.

In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)
#import preprocessor
from html import unescape
import re
import string
import wordsegment as ws
#import emoji
ws.load() # load vocab for word segmentation

random_seed = 42

In [2]:
def regex_match_segmentation(match):
    return ' '.join(ws.segment(match.group(0)))
# Define function for cleaning text
def clean_text(text):
    
    # convert HTML codes
    text = unescape(text)
    
    # lowercase text
    text = text.lower()
    
    # replace mentions, URLs and emojis with special token
    text = re.sub(r"@[A-Za-z0-9_-]+",'[USER]',text)
    text = re.sub(r"u/[A-Za-z0-9_-]+",'[USER]',text)
    text = re.sub(r"http\S+",'[URL]',text)
    
    # find and split hashtags into words
    text = re.sub(r"#[A-Za-z0-9]+", regex_match_segmentation, text)

    # remove punctuation at beginning of string (quirk in Davidson data)
    text = text.lstrip("!")
    text = text.lstrip(":")
    
    # remove newline and tab characters
    text = text.replace('\n',' ')
    text = text.replace('\t',' ')
    text = text.replace('[linebreak]', ' ')
    
    return text

## CAD

In [6]:
pwd

'/Users/joanna/Documents/mva/S2/NLP/necessity-sufficiency'

In [9]:
cad_train = pd.read_csv("cad_naacl2021/cad_v1_1_train.tsv", sep="\t")
cad_valid = pd.read_csv("cad_naacl2021/cad_v1_1_dev.tsv", sep="\t")
cad_test = pd.read_csv("cad_naacl2021/cad_v1_1_test.tsv", sep="\t")

dfs = [cad_train, cad_valid, cad_test]

for dd in dfs: 
    dd.text = dd.text.astype(str).apply(lambda tt: clean_text(tt))
    dd.labels = dd.labels.apply(lambda x: x.split(','))
    
cad_train_hateful = cad_train.copy()
cad_valid_hateful = cad_valid.copy()
cad_test_hateful = cad_test.copy()

cad_train_abusive = cad_train.copy()
cad_valid_abusive = cad_valid.copy()
cad_test_abusive = cad_test.copy()

hate_dfs = [cad_train_hateful, cad_valid_hateful, cad_test_hateful]
abuse_dfs = [cad_train_abusive, cad_valid_abusive, cad_test_abusive]

for dd, oo in zip(hate_dfs, dfs):
    dd['label'] = oo.labels.apply(lambda x: 1 if 'IdentityDirectedAbuse' in x else 0)
    dd = dd.drop(columns=['labels'])
    
for dd, oo in zip(abuse_dfs, dfs):
    dd['label'] = oo.labels.apply(lambda x: 0 if ('Neutral' in x) or ('CounterSpeech' in x) else 1)
    dd = dd.drop(columns=['labels'])

In [10]:
cad_train_abusive.groupby('label').size().reset_index(name='counts')

,label,counts
0,0,11186
1,1,2398


In [11]:
cad_train_hateful.groupby('label').size().reset_index(name='counts')

,label,counts
0,0,12227
1,1,1357


In [18]:
cad_train_abusive.to_csv('Data/CAD_abuse/train.csv')
cad_valid_abusive.to_csv('Data/CAD_abuse/valid.csv')
cad_test_abusive.to_csv('Data/CAD_abuse/test.csv')

cad_train_hateful.to_csv('Data/CAD_hate/train.csv')
cad_valid_hateful.to_csv('Data/CAD_hate/valid.csv')
cad_test_hateful.to_csv('Data/CAD_hate/test.csv')

## Founta

For this dataset, we've already created a train-valid-test split while prepping the data for ILM. 

In [25]:
founta_train = pd.read_csv("Data/Founta/train.csv", index_col=0)
founta_valid = pd.read_csv("Data/Founta/valid.csv",index_col=0)
founta_test = pd.read_csv("Data/Founta/test.csv", index_col=0)

dfs = [founta_train, founta_valid, founta_test]

for dd in dfs:
    dd.drop(dd[dd.label == 'spam'].index, inplace=True)
    dd.text = dd.text.astype(str).apply(lambda tt: clean_text(tt))
    dd = dd.drop(columns=['count_label_votes'])

founta_train_abusive = founta_train.copy()
founta_valid_abusive = founta_valid.copy()
founta_test_abusive = founta_test.copy()

founta_train_hateful = founta_train.copy()
founta_valid_hateful = founta_valid.copy()
founta_test_hateful = founta_test.copy()

abuse_dfs = [founta_train_abusive, founta_valid_abusive, founta_test_abusive]
hate_dfs = [founta_train_hateful, founta_valid_hateful, founta_test_hateful]

for dd in abuse_dfs:
    dd.label.replace({'hateful': 1, "abusive": 1, "normal": 0}, inplace = True)

for dd in hate_dfs:
    dd.label.replace({'hateful': 1, "abusive": 0, "normal": 0}, inplace = True)


In [26]:
founta_train_abusive.groupby('label').size().reset_index(name='counts')

,label,counts
0,0,42588
1,1,19857


In [27]:
founta_train_hateful.groupby('label').size().reset_index(name='counts')

,label,counts
0,0,59150
1,1,3295


In [37]:
founta_train_abusive.to_csv('Data/Founta_abuse/train.csv')
founta_valid_abusive.to_csv('Data/Founta_abuse/valid.csv')
founta_test_abusive.to_csv('Data/Founta_abuse/test.csv')

founta_train_hateful.to_csv('Data/Founta_hate/train.csv')
founta_valid_hateful.to_csv('Data/Founta_hate/valid.csv')
founta_test_hateful.to_csv('Data/Founta_hate/test.csv')

## Davidson

In [12]:
davidson2017 = pd.read_csv('./Data/davidson2017.csv', index_col=0)
davidson2017.rename(columns={"class": "label", "tweet": "text"}, inplace=True, errors='ignore')
davidson2017 = davidson2017[['text','label']]
davidson2017.text = davidson2017.text.astype(str).apply(lambda tt: clean_text(tt))
davidson2017.label.replace({0: "hateful", 1: "offensive", 2: "neither"}, inplace = True)

davidson_train, davidson_valtest = train_test_split(davidson2017, 
                                                    test_size=0.2, 
                                                    stratify=davidson2017.label, 
                                                    random_state=123)
davidson_valid, davidson_test = train_test_split(davidson_valtest, 
                                                 test_size=0.5, 
                                                 stratify=davidson_valtest.label, 
                                                 random_state=123)

d_train_offense = davidson_train.copy()
d_valid_offense = davidson_valid.copy()
d_test_offense = davidson_test.copy()

d_train_hate = davidson_train.copy()
d_valid_hate = davidson_valid.copy()
d_test_hate = davidson_test.copy()

dfs_offense = [d_train_offense, d_valid_offense, d_test_offense]
dfs_hate = [d_train_hate, d_valid_hate, d_test_hate]

for dd in dfs_offense:
    dd.label.replace({'hateful': 1, 'offensive': 1, 'neither': 0}, inplace = True)
    
for dd in dfs_hate: 
    dd.label.replace({'hateful': 1, 'offensive': 0, 'neither': 0}, inplace = True)

/var/folders/2q/yvjnzdxd1b9bjpjqhmjv1qq00000gn/T/ipykernel_3306/3089464611.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  davidson2017.label.replace({0: "hateful", 1: "offensive", 2: "neither"}, inplace = True)
/var/folders/2q/yvjnzdxd1b9bjpjqhmjv1qq00000gn/T/ipykernel_3306/3089464611.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate obj

In [13]:
d_train_offense.groupby('label').size().reset_index(name='counts')

,label,counts
0,0,3330
1,1,16496


In [14]:
d_train_hate.groupby('label').size().reset_index(name='counts')

,label,counts
0,0,18682
1,1,1144


In [16]:
d_train_offense.to_csv("Data/Davidson_abuse/train.csv")
d_valid_offense.to_csv("Data/Davidson_abuse/valid.csv")
d_test_offense.to_csv("Data/Davidson_abuse/test.csv")

d_train_hate.to_csv("Data/Davidson_hate/train.csv")
d_valid_hate.to_csv("Data/Davidson_hate/valid.csv")
d_test_offense.to_csv("Data/Davidson_hate/test.csv")